In [1]:
## 使用Unstructured File读取text文件、powerpoints、html、PDF、images或其他需要安装以下
## windows安装detectron诸多问题，可参考：
#####  - https://blog.csdn.net/u010674979/article/details/125719919
#####  - http://www.taodudu.cc/news/show-1215672.html

# # Install package
# !pip install "unstructured[local-inference]"
# !pip install "detectron2@git+https://github.com/facebookresearch/detectron2.git@v0.6#egg=detectron2"
# !pip install layoutparser[layoutmodels,tesseract]

In [2]:
# # Install other dependencies
# # https://github.com/Unstructured-IO/unstructured/blob/main/docs/source/installing.rst
# !brew install libmagic
# !brew install poppler
# !brew install tesseract
# # If parsing xml / html documents:
# !brew install libxml2
# !brew install libxslt

'brew' is not recognized as an internal or external command,
operable program or batch file.
'brew' is not recognized as an internal or external command,
operable program or batch file.
'brew' is not recognized as an internal or external command,
operable program or batch file.
'brew' is not recognized as an internal or external command,
operable program or batch file.
'brew' is not recognized as an internal or external command,
operable program or batch file.


### 自定义数据源读取

In [1]:
!pwd

/f/AI/hohoChat/debug


In [2]:
cd ../langchain-ChatGLM-master/

F:\AI\hohoChat\langchain-ChatGLM-master


In [3]:
!pwd

/f/AI/hohoChat/langchain-ChatGLM-master


In [4]:
import torch.cuda
import torch.backends

from configs import model_config
from models.chatglm_llm import ChatGLM
from chains.local_doc_qa import *
from utils import torch_gc

In [5]:
# !pip install PyPDF2

In [6]:
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import ElasticVectorSearch, Pinecone, Weaviate, FAISS
from langchain.chains.question_answering import load_qa_chain

In [8]:
reader = PdfReader('../data/d2l-zh-pytorch.pdf')

In [9]:
raw_text = ""
for i, page in enumerate(reader.pages):
    text = page.extract_text()
    if text:
        raw_text += text

In [11]:
raw_text[:100]

'动⼿学深度学习\nRelease2.0.0\nAstonZhang,ZacharyC.Lipton,MuLi,andAlexanderJ.Smola\nFeb09,2023⽬录\n前⾔ 1\n安装 9\n符号 1'

In [12]:
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = model_config.CHUNK_SIZE,
    chunk_overlap = 30,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [13]:
len(texts)

4874

In [14]:
texts[0]

'动⼿学深度学习\nRelease2.0.0\nAstonZhang,ZacharyC.Lipton,MuLi,andAlexanderJ.Smola\nFeb09,2023⽬录\n前⾔ 1\n安装 9\n符号 13\n1引⾔ 17\n2预备知识 39'

In [15]:
texts[1]

'安装 9\n符号 13\n1引⾔ 17\n2预备知识 39\n2.1数据操作 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 40'

In [16]:
embeddings = HuggingFaceEmbeddings(model_name = model_config.embedding_model_dict[model_config.EMBEDDING_MODEL], 
                                   model_kwargs = {'device': model_config.EMBEDDING_DEVICE})

No sentence-transformers model found with name C:\Users\Administrator/.cache\torch\sentence_transformers\GanymedeNil_text2vec-large-chinese. Creating a new one with MEAN pooling.


In [17]:
vector_store = FAISS.from_texts(texts, embeddings)
# vector_store = FAISS.from_documents(texts, embeddings)

FAISS.similarity_search_with_score_by_vector = similarity_search_with_score_by_vector
vector_store.chunk_size = model_config.CHUNK_SIZE

In [16]:
# torch_gc(model_config.EMBEDDING_DEVICE)

In [18]:
llm = ChatGLM()
llm.load_model(model_name_or_path = model_config.llm_model_dict[model_config.LLM_MODEL],
               llm_device = model_config.LLM_DEVICE,
               use_ptuning_v2 = model_config.USE_PTUNING_V2)
llm.history_len = LLM_HISTORY_LEN

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


No compiled kernel found.
Compiling kernels : C:\Users\Administrator\.cache\huggingface\modules\transformers_modules\THUDM\chatglm-6b-int4\d8a6cfc6cb4d5b0fbc51183e26c06f4a3cb26947\quantization_kernels_parallel.c
Compiling gcc -O3 -fPIC -pthread -fopenmp -std=c99 C:\Users\Administrator\.cache\huggingface\modules\transformers_modules\THUDM\chatglm-6b-int4\d8a6cfc6cb4d5b0fbc51183e26c06f4a3cb26947\quantization_kernels_parallel.c -shared -o C:\Users\Administrator\.cache\huggingface\modules\transformers_modules\THUDM\chatglm-6b-int4\d8a6cfc6cb4d5b0fbc51183e26c06f4a3cb26947\quantization_kernels_parallel.so
Compile default cpu kernel failed, using default cpu kernel code.
Compiling gcc -O3 -fPIC -std=c99 C:\Users\Administrator\.cache\huggingface\modules\transformers_modules\THUDM\chatglm-6b-int4\d8a6cfc6cb4d5b0fbc51183e26c06f4a3cb26947\quantization_kernels.c -shared -o C:\Users\Administrator\.cache\huggingface\modules\transformers_modules\THUDM\chatglm-6b-int4\d8a6cfc6cb4d5b0fbc51183e26c06f4a3

In [19]:
def answer_based_on_knowledge(query, vector_store, llm, chat_history = []):
    related_docs_with_score = vector_store.similarity_search_with_score(query, k = VECTOR_SEARCH_TOP_K)
    related_docs = get_docs_with_score(related_docs_with_score)
    prompt = generate_prompt(related_docs, query)

    # if streaming:
    #     for result, history in self.llm._stream_call(prompt=prompt,
    #                                                  history=chat_history):
    #         history[-1][0] = query
    #         response = {"query": query,
    #                     "result": result,
    #                     "source_documents": related_docs}
    #         yield response, history
    # else:
    for result, history in llm._call(prompt = prompt, history = chat_history, streaming = STREAMING):
        history[-1][0] = query
        response = {"query": query,
                    "result": result,
                    "source_documents": related_docs}
        yield response, history


In [20]:
from IPython.display import display, Markdown, clear_output

def display_answer(query, chat_history = []):
    for resp, history in answer_based_on_knowledge(query, vector_store, llm, chat_history):
        clear_output(wait = True)
        display(Markdown(resp['result']))
    
    return resp, history

In [21]:
resp, chat_history = display_answer("什么是DenseNet模型?")

The dtype of attention mask (torch.int64) is not bool


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ C:\Users\Administrator\AppData\Local\Temp\ipykernel_147228\1034128721.py:1 in <module>           │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ 'C:\\Users\\Administrator\\AppData\\Local\\Temp\\ipykernel_147228\\1034128721.py'                │
│                                                                                                  │
│ C:\Users\Administrator\AppData\Local\Temp\ipykernel_147228\1378288236.py:4 in display_answer     │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ 'C:\\Users\\Administrator\\AppData\\Local\\Temp\\ipykernel_147228\\1378288236.py'                │
│                                                                                                  │
│ C:\Users\Administrator\AppData\Local\Temp\ipykernel_147228\230243537.py:15 in                    │
│ answer_based_on_knowledge                                                                        │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ 'C:\\Users\\Administrator\\AppData\\Local\\Temp\\ipykernel_147228\\230243537.py'                 │
│                                                                                                  │
│ F:\AI\hohoChat\langchain-ChatGLM-master\models\chatglm_llm.py:79 in _call                        │
│                                                                                                  │
│    76 │   │   │   │   │   history[-1] = [prompt, stream_resp]                                    │
│    77 │   │   │   │   yield stream_resp, history                                                 │
│    78 │   │   else:                                                                              │
│ ❱  79 │   │   │   response, _ = self.model.chat(                                                 │
│    80 │   │   │   │   │   self.tokenizer,                                                        │
│    81 │   │   │   │   │   prompt,                                                                │
│    82 │   │   │   │   │   history=history[-self.history_len:] if self.history_len > 0 else [],   │
│                                                                                                  │
│ D:\ProgramData\Anaconda3\envs\hoho_chatgpt\lib\site-packages\torch\autograd\grad_mode.py:27 in   │
│ decorate_context                                                                                 │
│                                                                                                  │
│    24 │   │   @functools.wraps(func)                                                             │
│    25 │   │   def decorate_context(*args, **kwargs):                                             │
│    26 │   │   │   with self.clone():                                                             │
│ ❱  27 │   │   │   │   return func(*args, **kwargs)                                               │
│    28 │   │   return cast(F, decorate_context)                                                   │
│    29 │                                                                                          │
│    30 │   def _wrap_generator(self, func):                                                       │
│                                                                                                  │
│ C:\Users\Administrator/.cache\huggingface\modules\transformers_modules\THUDM\chatglm-6b-int4\d8a │
│ 6cfc6cb4d5b0fbc51183e26c06f4a3cb26947\modeling_chatglm.py:1

In [20]:
resp

{'query': 'who are the authors of the article',
 'result': 'The article does not provide足够的相关信息 about the authors.',
 'source_documents': [Document(page_content='stories. We designed prompt templates to create\ndifferent scenarios for creative writing. The cre-\native prompt template was inspired by Mad Libs\nstyle variations of ‘Write a [creative story type]\nabout [NOUN] in the style of [PERSON]‘. In ear-\nlier versions of GPT4All, we found that rather than\nwriting actual creative content, the model would\ndiscuss how it would go about writing the content.\nTraining on this new dataset allows GPT4All-J to\nwrite poems, songs, and plays with increased com-', metadata={'score': 779.44946}),
  Document(page_content='to that produced by burning 20 gallons (75 liters)\nof gasoline.\n3 Evaluation\nWe perform a preliminary evaluation of our model\nusing the human evaluation data from the Self-\nInstruct paper (Wang et al., 2022). We report the\nground truth perplexity of our model against 

In [21]:
chat_history

[['who are the authors of the article',
  'The article does not provide足够的相关信息 about the authors.']]

In [22]:
resp, chat_history = display_answer("what was the cost of training GPT4all model?", chat_history = [])

训练 GPT4All 模型的成本包括：数据收集和处理、代码编写和测试。具体来说，我们花费了大量时间和精力来收集和处理数据，并编写了训练和测试代码。

In [23]:
resp

{'query': 'what was the cost of training GPT4all model?',
 'result': '训练 GPT4All 模型的成本包括：数据收集和处理、代码编写和测试。具体来说，我们花费了大量时间和精力来收集和处理数据，并编写了训练和测试代码。',
 'source_documents': [Document(page_content='GPT4All-J dataset that is a superset of the origi-\nnal 400k points GPT4All dataset. We dedicated\nsubstantial attention to data preparation and cura-\ntion.Building on the GPT4All dataset, we curated\nthe GPT4All-J dataset by augmenting the origi-\nnal 400k GPT4All examples with new samples\nencompassing additional multi-turn QA samples\nand creative writing such as poetry, rap, and short\nstories. We designed prompt templates to create\ndifferent scenarios for creative writing. The cre-stories. We designed prompt templates to create\ndifferent scenarios for creative writing. The cre-\native prompt template was inspired by Mad Libs\nstyle variations of ‘Write a [creative story type]\nabout [NOUN] in the style of [PERSON]‘. In ear-\nlier versions of GPT4All, we found that rather than\nwriting actua

In [24]:
chat_history

[['what was the cost of training GPT4all model?',
  '训练 GPT4All 模型的成本包括：数据收集和处理、代码编写和测试。具体来说，我们花费了大量时间和精力来收集和处理数据，并编写了训练和测试代码。']]

In [25]:
resp, chat_history = display_answer("模型是如何被训练的?", chat_history = [])

GPT4All是一个基于多轮对话、代码、诗歌、歌曲和故事的人工智能助手。它通过训练更大的语料库，使用Apache- licensed GPT-J模型而不是GPL- licensed LLaMA模型，以及在创造性任务(如写作诗歌、歌曲和剧本)中表现出更好的性能来构建。我们公开发布训练数据、数据 curated 过程、训练代码和最终模型权重，以促进开放的研究故事。我们设计了一些 prompt 模板，用于创建不同的创造性写作场景。这些创造性 prompt 模板的灵感来自于《Madlibs》风格的模板，以“写一个[创造性故事类型]关于[noun]的风格[人物]”为例。在早期的 GPT4All 版本中，我们发现，模型并不真正编写实际创造性内容，而是讨论如何编写这些内容。通过训练使用这个新语料库，GPT4All-J 可以写更多的诗歌、歌曲和剧本，与个人健康和 wellness 相关的几代人联系起来。

In [27]:
resp, chat_history = display_answer("what was the size of training dataset?", chat_history = [])

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ C:\Users\Administrator\AppData\Local\Temp\ipykernel_142464\4064121317.py:1 in <module>           │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ 'C:\\Users\\Administrator\\AppData\\Local\\Temp\\ipykernel_142464\\4064121317.py'                │
│                                                                                                  │
│ C:\Users\Administrator\AppData\Local\Temp\ipykernel_142464\1378288236.py:4 in display_answer     │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ 'C:\\Users\\Administrator\\AppData\\Local\\Temp\\ipykernel_142464\\1378288236.py'                │
│                                                                                                  │
│ C:\Users\Administrator\AppData\Local\Temp\ipykernel_142464\230243537.py:15 in                    │
│ answer_based_on_knowledge                                                                        │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ 'C:\\Users\\Administrator\\AppData\\Local\\Temp\\ipykernel_142464\\230243537.py'                 │
│                                                                                                  │
│ F:\AI\hohoChat\langchain-ChatGLM-master\models\chatglm_llm.py:79 in _call                        │
│                                                                                                  │
│    76 │   │   │   │   │   history[-1] = [prompt, stream_resp]                                    │
│    77 │   │   │   │   yield stream_resp, history                                                 │
│    78 │   │   else:                                                                              │
│ ❱  79 │   │   │   response, _ = self.model.chat(                                                 │
│    80 │   │   │   │   │   self.tokenizer,                                                        │
│    81 │   │   │   │   │   prompt,                                                                │
│    82 │   │   │   │   │   history=history[-self.history_len:] if self.history_len > 0 else [],   │
│                                                                                                  │
│ D:\ProgramData\Anaconda3\envs\hoho_chatgpt\lib\site-packages\torch\autograd\grad_mode.py:27 in   │
│ decorate_context                                                                                 │
│                                                                                                  │
│    24 │   │   @functools.wraps(func)                                                             │
│    25 │   │   def decorate_context(*args, **kwargs):                                             │
│    26 │   │   │   with self.clone():                                                             │
│ ❱  27 │   │   │   │   return func(*args, **kwargs)                                               │
│    28 │   │   return cast(F, decorate_context)                                                   │
│    29 │                                                                                          │
│    30 │   def _wrap_generator(self, func):                                                       │
│                                                                                                  │
│ C:\Users\Administrator/.cache\huggingface\modules\transformers_modules\THUDM\chatglm-6b-int4\d8a │
│ 6cfc6cb4d5b0fbc51183e26c06f4a3cb26947\modeling_chatglm.py:1